## Error Analysis

In [1]:
import json
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import classification_report
from tqdm import tqdm
tqdm.pandas()

In [2]:
def collect_predictions(df, folder, model_names, file):
    for model_name in model_names:
        df[model_name] = pd.read_csv(f"0_resources/3_model_predictions/baselines/{model_name}/{file}")[model_name]
    return df

In [3]:
baseline_models = [
    ('twitter-xlm-roberta-base',  'XLMT'),
    ('indobertweet-base-uncased', 'IndoT')
]

In [4]:
testset = pd.read_csv("0_resources/0_dataset/test_split.csv")
testset = collect_predictions(testset, 'baselines', list(zip(*baseline_models))[0], 'test_set_predictions.csv')

In [5]:
attacks = len(testset[testset['label'] == 1])
print(f"{attacks:,.0f} ({(attacks/len(testset))*100:.2f}%) comments were majority-labelled as containing attacks.")

954 (31.80%) comments were majority-labelled as containing attacks.


### Subreddit

In [6]:
SUBREDDIT_CUTOFF = 5

In [7]:
subreddits = pd.DataFrame(
    {
        'Comments': testset['subreddit'].value_counts(),
        '% of Data': testset['subreddit'].value_counts(normalize=True)*100
    },
    index=testset['subreddit'].value_counts().index
)
other_count = np.sum(subreddits[subreddits['Comments'] < SUBREDDIT_CUTOFF].Comments)
others = list(subreddits[subreddits['Comments'] < SUBREDDIT_CUTOFF].index)
subreddits = subreddits[subreddits['Comments'] > SUBREDDIT_CUTOFF]

subreddits.loc['Other'] = [other_count,  other_count/30]

#### Calculate % of Attacks

In [8]:
def calculate_pct_by_grouper(df, col_to_group_by, target_col='label'):
    raw = df.groupby([col_to_group_by, target_col]).id.count().unstack(fill_value=0).stack()
    summary_df = pd.DataFrame(raw, columns=['Comments'])
    total = df.groupby([col_to_group_by]).id.count()
    summary_df['% of Subreddit'] = raw.div(total, level=col_to_group_by) * 100
    return summary_df

In [9]:
attack_summary_df = calculate_pct_by_grouper(testset, 'subreddit')
attacks = []
for idx, row in subreddits.iterrows():
    try:
        attacks.append(attack_summary_df.loc[idx].loc[1]['% of Subreddit'])
    except:
        other_attacks = []
        for other_subreddit in others:
            other_attacks.append(attack_summary_df.loc[other_subreddit].loc[1]['% of Subreddit'])
        attacks.append(np.mean(other_attacks))    
            
subreddits['% Attacks'] = attacks

#### Calculate % of Indonesian Comments

In [10]:
indonesian = []
for idx, row in subreddits.iterrows():
    if idx in list(testset.subreddit):
        indonesian.append((len(testset[(testset['subreddit']==idx) & (testset['final_label_language']=='Indonesian')]['label'])/len(testset[testset['subreddit']==idx]))*100)
    else:
        indonesian.append((len(testset[(testset['subreddit'].isin(others)) & (testset['final_label_language']=='Indonesian')]['label'])/len(testset[testset['subreddit'].isin(others)]))*100)
subreddits['% Indonesian'] = indonesian

#### Calculate Error Rate

In [11]:
CLASSIFICATION_CUTOFF = 0.5

In [12]:
def calculate_error_rate(gold_labels, predictions):
    report = classification_report(gold_labels, label(predictions), digits=4, output_dict=True)
    return (report['accuracy']*100, report['1']['precision']*100, report['1']['recall']*100, report['macro avg']['f1-score']*100)    

def label(predictions)->[]:
    return [1 if p > CLASSIFICATION_CUTOFF else 0 for p in predictions] 

In [13]:
for model_name, model_description in baseline_models:
    error_analysis = []
    for idx, row in subreddits.iterrows():
        if idx in list(testset.subreddit):
            gold_labels = testset[testset['subreddit']==idx]['label']
            predictions = testset[testset['subreddit']==idx][model_name]
            error_analysis.append(calculate_error_rate(gold_labels, predictions))
        else:
            gold_labels = testset[testset['subreddit'].isin(others)]['label']
            predictions = testset[testset['subreddit'].isin(others)][model_name]
            error_analysis.append(calculate_error_rate(gold_labels, predictions))
    for column_name, entries in zip(['Acc', 'Prec', 'Rec', 'Macro F1'], list(zip(*error_analysis))):
        subreddits[model_description + ' ' + column_name] = entries

In [14]:
pd.options.display.float_format = "{:,.1f}".format
subreddits

,Comments,% of Data,% Attacks,% Indonesian,XLMT Acc,XLMT Prec,XLMT Rec,XLMT Macro F1,IndoT Acc,IndoT Prec,IndoT Rec,IndoT Macro F1
indonesia,"2,539.0",84.6,30.6,94.0,81.1,70.7,65.5,77.3,82.7,74.5,65.9,78.9
malaysia,254.0,8.5,41.7,20.5,80.3,74.6,80.2,80.0,82.7,77.7,82.1,82.3
singapore,49.0,1.6,20.4,14.3,85.7,66.7,60.0,77.1,85.7,66.7,60.0,77.1
malaygonewild,43.0,1.4,51.2,18.6,81.4,76.9,90.9,81.1,67.4,63.3,86.4,66.0
MalaysGoneWild,40.0,1.3,37.5,22.5,82.5,75.0,80.0,81.6,87.5,77.8,93.3,87.1
Ajar_Malaysia,18.0,0.6,27.8,11.1,83.3,66.7,80.0,80.4,83.3,75.0,60.0,77.8
MalaysianFappers,11.0,0.4,45.5,9.1,63.6,57.1,80.0,63.3,36.4,33.3,40.0,36.4
malaysians,10.0,0.3,30.0,30.0,70.0,50.0,33.3,60.0,70.0,50.0,33.3,60.0
NegarakuMalaysia,6.0,0.2,16.7,16.7,66.7,33.3,100.0,62.5,66.7,33.3,100.0,62.5
Other,30.0,1.0,34.8,20.0,76.7,63.6,70.0,74.4,70.0,54.5,60.0,67.0


### Language

In [15]:
LANGUAGE_CUTOFF = 10

In [16]:
languages = pd.DataFrame(
    {
        'Comments': testset['final_label_language'].value_counts(),
        '% of Data': testset['final_label_language'].value_counts(normalize=True)*100
    },
    index=testset['final_label_language'].value_counts().index
)
other_count = np.sum(languages[languages['Comments'] < LANGUAGE_CUTOFF].Comments)
others = list(languages[languages['Comments'] < LANGUAGE_CUTOFF].index)
languages = languages[languages['Comments'] > LANGUAGE_CUTOFF]

languages.loc['Other Languages'] = [other_count,  other_count/30]

#### Calculate % of Attacks

In [17]:
attack_summary_df = calculate_pct_by_grouper(testset, 'final_label_language')
attacks = []
for idx, row in languages.iterrows():
    try:
        attacks.append(attack_summary_df.loc[idx].loc[1]['% of Subreddit'])
    except:
        other_attacks = []
        for other_subreddit in others:
            other_attacks.append(attack_summary_df.loc[other_subreddit].loc[1]['% of Subreddit'])
        attacks.append(np.mean(other_attacks))  
            
languages['% Attacks'] = attacks

#### Calculate Error Rate

In [18]:
for model_name, model_description in baseline_models:
    error_analysis = []
    for idx, row in languages.iterrows():
        if idx in list(testset.final_label_language):
            gold_labels = testset[testset['final_label_language']==idx]['label']
            predictions = testset[testset['final_label_language']==idx][model_name]
            error_analysis.append(calculate_error_rate(gold_labels, predictions))
        else:
            gold_labels = testset[testset['final_label_language'].isin(others)]['label']
            predictions = testset[testset['final_label_language'].isin(others)][model_name]
            error_analysis.append(calculate_error_rate(gold_labels, predictions))
    for column_name, entries in zip(['Acc', 'Prec', 'Rec', 'Macro F1'], list(zip(*error_analysis))):
        languages[model_description + ' ' + column_name] = entries

In [19]:
pd.options.display.float_format = "{:,.1f}".format
languages

,Comments,% of Data,% Attacks,XLMT Acc,XLMT Prec,XLMT Rec,XLMT Macro F1,IndoT Acc,IndoT Prec,IndoT Rec,IndoT Macro F1
Indonesian,"2,476.0",82.5,30.5,81.0,70.2,65.3,77.1,82.7,74.1,66.3,78.9
Malay,276.0,9.2,48.6,80.1,76.5,85.1,80.1,78.6,74.5,85.1,78.6
"('English', 'Indonesian')",94.0,3.1,25.5,80.9,63.6,58.3,74.1,81.9,68.4,54.2,74.4
Singlish,38.0,1.3,28.9,76.3,58.3,63.6,71.9,76.3,58.3,63.6,71.9
English,37.0,1.2,2.7,100.0,100.0,100.0,100.0,97.3,50.0,100.0,82.6
"('English', 'Malay')",19.0,0.6,47.4,84.2,80.0,88.9,84.2,73.7,83.3,55.6,72.5
Other Languages,50.0,1.7,26.4,80.0,71.4,62.5,76.2,80.0,80.0,50.0,74.0


#### Qualitative Model Comparison

In [20]:
comparison = len(
    testset[
        (testset['final_label_language'] == 'Indonesian') &
        (testset['label'] == 1 ) &
        (testset['twitter-xlm-roberta-base'] < 0.5) &
        (testset['indobertweet-base-uncased'] > 0.5)
    ][['text', 'twitter-xlm-roberta-base', 'indobertweet-base-uncased']].index
)

In [21]:
print(f"Indonesian Attacks correctly identified by IndoBERTweet but misclassified by XLM-T: {comparison:,.0f}")

Indonesian Attacks correctly identified by IndoBERTweet but misclassified by XLM-T: 78


In [22]:
print(f"Example: {testset.iloc[26].text}")

Example: Jadi gini mbak, rasanya kontol saya pengen saya cekek deh liat mbak soalnya mbak ngomongnya dah kek kontol


### Attack Type

In [25]:
testset = pd.read_csv("0_resources/0_dataset/test_split.csv")
testset = collect_predictions(testset, 'baselines', list(zip(*baseline_models))[0], 'test_set_predictions.csv')
full_data = pd.read_csv("0_resources/0_dataset/full.csv")

In [26]:
testset = pd.merge(testset, full_data, on="id")

In [27]:
def convert_to_majority(entry):
    try:
        entry = json.loads(entry.replace('nan', "'nan'").replace('\'', "\""))
        if Counter(entry)['Yes'] > 1:
            return 1
    except: 
        if (entry.count('[nan,')+entry.count(' nan,')+entry.count('nan]')) < 1:
            return 1
    return 0

In [28]:
attack_types = [
    ('attack_person', 'Attack on a Person'),
    ('attack_media', 'Attack on Media'), 
    ('attack_protected', 'Attack on a Protected Group'),
    ('attack_institution', 'Attack on an Institution'), 
    ('attack_other', 'Other Attacks')
]

In [29]:
for attack_type, attack_description in attack_types:
    testset[attack_type + "_maj"] = testset[attack_type].progress_apply(lambda x: convert_to_majority(x))

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 402099.89it/s]


#### Calculate Error Rate

In [30]:
import warnings
warnings.filterwarnings('ignore')

attack_type_analysis = []
for attack_type, attack_description in attack_types:
    
    results = [attack_description]
    entries = testset[testset[attack_type + "_maj"] == 1]
    results.append(len(entries))
    results.append((len(entries)/len(testset))*100)
    
    gold_labels = entries['label']
    for model_name, model_description in baseline_models:
        predictions = entries[model_name]      
        a,_,_,_ = calculate_error_rate(gold_labels, predictions)
        results.append(a)

    attack_type_analysis.append(results)

df_columns = ['Attack Type', 'Comments', '% of Data']
for model_name, model_description in baseline_models:
    df_columns.append(f'{model_description} Acc')

attack_type_analysis = pd.DataFrame(attack_type_analysis, columns=df_columns)

In [31]:
attack_type_analysis.set_index('Attack Type')

,Comments,% of Data,XLMT Acc,IndoT Acc
Attack Type,,,,
Attack on a Person,679,22.6,70.3,72.5
Attack on Media,11,0.4,72.7,63.6
Attack on a Protected Group,81,2.7,70.4,65.4
Attack on an Institution,69,2.3,78.3,73.9
Other Attacks,4,0.1,75.0,100.0


#### Qualitative Model Comparison

In [32]:
comparison = len(
    list(testset[
        (testset['attack_institution_maj'] == 1) &
        (testset['label'] == 1) &
        (testset['twitter-xlm-roberta-base'] > 0.5) &
        (testset['indobertweet-base-uncased'] < 0.5)]['text_x']
    )
)

In [33]:
print(f"INSTITUTION Attacks correctly identified by XLM-T but misclassified by IndoBERTweet: {comparison:,.0f}")

INSTITUTION Attacks correctly identified by XLM-T but misclassified by IndoBERTweet: 9


In [34]:
comparison = len(
    list(testset[
        (testset['attack_person_maj'] == 1) &
        (testset['label'] == 1) &
        (testset['twitter-xlm-roberta-base'] < 0.5) &
        (testset['indobertweet-base-uncased'] > 0.5)]['text_x']
    )
)

In [35]:
print(f"PERSON Attacks correctly identified by IndoBERTweet but misclassified by XLM-T: {comparison:,.0f}")

PERSON Attacks correctly identified by IndoBERTweet but misclassified by XLM-T: 69
